In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [18]:
driver = webdriver.Firefox()
driver.get("https://animepahe.ru")

In [19]:
input = driver.find_element(By.CLASS_NAME, "input-search")
input.send_keys("Jujutsu kaisen")

In [20]:
# Get search results
search_result_wrap = driver.find_element(By.CLASS_NAME, "search-results-wrap")

while True:
    try:
        ul =  search_result_wrap.find_element(By.CLASS_NAME, "search-results")
        li = ul.find_elements(By.TAG_NAME, "li")
        for anime in li:
            a = anime.find_element(By.TAG_NAME, "a")
            result_title = a.find_element(By.CLASS_NAME, "result-title")
            print (result_title.text.strip())
        break
    except Exception as e:
        print (f"Couldn't find the results. Retrying in 5s: {e}")
        time.sleep(5)

Couldn't find the results. Retrying in 5s: Message: Unable to locate element: .search-results; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:507:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16

Jujutsu Kaisen: Shimetsu Kaiyuu
Jujutsu Kaisen
Jujutsu Kaisen 2nd Season
Jujutsu Kaisen 0 Movie
Shenmue the Animation


In [22]:
import requests

response = requests.get("https://animepahe.ru/api?m=search&q=Jujutsu%20kaisen")
jsonres = None

if response.status_code == 200:
    # Print the response content (usually in JSON format for APIs)
    jsonres = response.json()
    print(jsonres)
else:
    # Print an error message if the request was not successful
    print(f"Error: {response.status_code} - {response.text}")

{'total': 5, 'per_page': 8, 'current_page': 1, 'last_page': 1, 'from': 1, 'to': 5, 'data': [{'id': 5469, 'title': 'Jujutsu Kaisen: Shimetsu Kaiyuu', 'type': 'TV', 'episodes': 0, 'status': 'Not yet aired', 'season': '', 'year': 0, 'score': None, 'poster': 'https://i.animepahe.ru/posters/9ba9ec2afa3c3408433d5973b10fc03d058553670fb6f4ca621fee0700efee8c.jpg', 'session': '957389cf-eb4a-defd-fae4-6aa0ad5206c8'}, {'id': 4072, 'title': 'Jujutsu Kaisen', 'type': 'TV', 'episodes': 24, 'status': 'Finished Airing', 'season': 'Fall', 'year': 2020, 'score': 8.62, 'poster': 'https://i.animepahe.ru/posters/19a68cbfed82c21304983b220f7d2ca5d870dd14f50229a63c390f4376711250.jpg', 'session': 'd36ba0bd-240d-b0b0-e260-f31d74c18482'}, {'id': 5262, 'title': 'Jujutsu Kaisen 2nd Season', 'type': 'TV', 'episodes': 23, 'status': 'Finished Airing', 'season': 'Summer', 'year': 2023, 'score': 8.91, 'poster': 'https://i.animepahe.ru/posters/32fb7e8b4a3dfc4fc76d2d2c6ab21cbc5c45b138d5949d3b746f9003a4e71fd1.jpg', 'sessio

In [29]:
# getting the second search result
driver = webdriver.Firefox()
driver.get(f"https://animepahe.ru/anime/{jsonres['data'][1]['session']}")

In [30]:
# getting the first episode
As = driver.find_elements(By.CSS_SELECTOR, ".episode-list-wrapper .play")
links = []
for a in As:
    links.append(a.get_attribute("href"))
print(links)

['https://animepahe.ru/play/d36ba0bd-240d-b0b0-e260-f31d74c18482/e63fd4fe65d6cc2dbbcf4e8c05e27cce945ebdce00a9c0510369babca964a0d2', 'https://animepahe.ru/play/d36ba0bd-240d-b0b0-e260-f31d74c18482/5e33b1b4d2fb945a70b3059a4578216799909a1920713694cceb5c40cf193c9d', 'https://animepahe.ru/play/d36ba0bd-240d-b0b0-e260-f31d74c18482/2789dcdf4b6035196b8fca4913f5fcf42ec96957a521145f2897dc6f1fd1f016', 'https://animepahe.ru/play/d36ba0bd-240d-b0b0-e260-f31d74c18482/581cba99092319e6caef324d4eeea70b5c9d3527d3f8ffa8cff51254b127ec3d', 'https://animepahe.ru/play/d36ba0bd-240d-b0b0-e260-f31d74c18482/016a98c1d4062b11602344805d69e8e5b6447850d934fe7704793f1c64eaed79', 'https://animepahe.ru/play/d36ba0bd-240d-b0b0-e260-f31d74c18482/0005553e48b50020e3b0bff95a2a6a34300ec6d214a6c958628bf19b0dd26df4', 'https://animepahe.ru/play/d36ba0bd-240d-b0b0-e260-f31d74c18482/8135293bf3ac3e66645e3459437f430a6e5521768adf20a699ff8567c5bb67b1', 'https://animepahe.ru/play/d36ba0bd-240d-b0b0-e260-f31d74c18482/81ba963158e7e95c61